In [1]:
#!pip install sentence_transformers
#!pip install newspaper3k

import torch

import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import time
import datetime
from datetime import datetime as DT
import json
import random
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import nltk
# nltk.download('punkt')
# nltk.download('brown')
# Locals 
from pathlib import Path 
import os, sys 
import glob

from IPython.display import Image
from IPython.core.display import HTML 
from sentence_transformers import SentenceTransformer, util

# nltk.download('punkt')
###
module_path = str(Path.cwd().parents[0] / "")
if module_path not in sys.path:
    sys.path.append(module_path)   
from utils import *
import random
from timeit import default_timer as timer
import time
import datetime
from datetime import datetime as DT
from os import listdir
import os.path as osp
from os.path import isfile, join

# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

import pandas as pd
import numpy as np
from dynamoDButils import *
# regex
import re
p = re.compile('[a-z]+')
p
re.compile('[a-z]+')
## LOCAL 
CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
from APIutils import *

###
def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')


Cell imports done @ 2023-01-19 18:18:15


In [2]:
jsonfile = join(JSONPath, "semanticSimArticlesTesting.json")

## demonstrate we can read the JSON file into a Pandas DataFrame
df = pd.read_json(jsonfile)
df
queriesdf = df.transpose() # pivot the matrix
queriesdf

,summary,scores
https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/,The Economist has created a visualization show...,"{'HF1': {'corpusindex': '5197', 'score': '0.56..."
https://venturebeat.com/ai/stability-ai-looks-to-aws-cloud-to-power-the-next-generation-of-generative-ai/,Among the organizations building generative AI...,"{'HF1': {'corpusindex': '6235', 'score': '0.99..."
https://venturebeat.com/ai/openai-ceo-admits-chatgpt-risks-what-now-the-ai-beat/,Watch on-demand sessions today.\nWatch HereAlt...,"{'HF1': {'corpusindex': '416', 'score': '0.576..."
https://venturebeat.com/ai/generative-ai-may-be-the-magic-potion-small-marketing-agencies-have-been-waiting-for/,Generative AI is going to completely alter the...,"{'HF1': {'corpusindex': '3961', 'score': '0.75..."
https://www.businessinsider.com/moderna-ceo-interview-on-mrnas-future-in-cancer-care-2022-12,Personalized cancer vaccines could provide a n...,"{'HF1': {'corpusindex': '2393', 'score': '0.56..."
...,...,...
https://www.foxbusiness.com/personal-finance/half-workers-dont-have-401k-retirement-savings,"If you’re struggling in the current economy, y...","{'HF1': {'corpusindex': '5918', 'score': '0.53..."
https://www.foxbusiness.com/politics/stuart-varney-theres-a-demand-for-a-new-generation-of-political-leaders,"During his latest ""My Take,"" ""Varney & Co."" ho...","{'HF1': {'corpusindex': '6181', 'score': '0.54..."
https://www.foxbusiness.com/lifestyle/serial-killer-john-wayne-gacys-clown-painting-sells-10k-spooky-auction,Famed serial killer John Wayne Gacy's creepy c...,"{'HF1': {'corpusindex': '3078', 'score': '0.29..."
https://www.foxbusiness.com/real-estate/us-suffering-second-biggest-home-price-correction-post-wwii-era,The U.S. housing market is experiencing its se...,"{'HF1': {'corpusindex': '546', 'score': '0.605..."


In [3]:
import json
import csv

with open(jsonfile) as json_file:
    data = json.load(json_file)

# data['https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/']    

keys= list(data.keys())
hf1 = data[keys[0]]['scores']['HF1']['score']
hf2 = data[keys[0]]['scores']['HF1']['score']
hf3 = data[keys[0]]['scores']['HF1']['score']
hf4 = data[keys[0]]['scores']['HF1']['score']
hf5 = data[keys[0]]['scores']['HF1']['score']

print (
f"hf1: {hf1} | hf2: {hf2} | hf3: {hf3} | hf4: {hf4} | hf5: {hf5} |"
)


hf1: 0.5627 | hf2: 0.5627 | hf3: 0.5627 | hf4: 0.5627 | hf5: 0.5627 |


In [4]:
def scoresDataframe(jsonfile):
    with open(jsonfile) as json_file:
        data = json.load(json_file)
    # data['https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/']    
    
    keys= list(data.keys()) 
    new_dict = {}
    article_url = []
    article_summary = []
    hf_score = {'hf1': [], 'hf2': [], 'hf3': [], 'hf4': [],'hf5': []}

    for indx in range(len(keys)):
        article_url.append(keys[indx])
        article_summary.append(data[keys[indx]]['summary'])
        hf_score['hf1'].append(data[keys[indx]]['scores']['HF1']['score'])
        hf_score['hf2'].append(data[keys[indx]]['scores']['HF2']['score'])
        hf_score['hf3'].append(data[keys[indx]]['scores']['HF3']['score'])
        hf_score['hf4'].append(data[keys[indx]]['scores']['HF4']['score'])
        hf_score['hf5'].append(data[keys[indx]]['scores']['HF5']['score'])

    new_dict = {
        "article": article_url, "summary": article_summary, "hf1": hf_score['hf1'], 
        "hf2": hf_score['hf2'], "hf3": hf_score['hf3'], "hf4": hf_score['hf4'], "hf5": hf_score['hf5']
    }
    return pd.DataFrame(new_dict)

In [5]:
df_test = scoresDataframe(jsonfile)
df_test

,article,summary,hf1,hf2,hf3,hf4,hf5
0,https://datainnovation.org/2022/11/visualizing...,The Economist has created a visualization show...,0.5627,0.5556,0.5153,0.5132,0.4939
1,https://venturebeat.com/ai/stability-ai-looks-...,Among the organizations building generative AI...,0.9986,0.6797,0.6685,0.6572,0.6319
2,https://venturebeat.com/ai/openai-ceo-admits-c...,Watch on-demand sessions today.\nWatch HereAlt...,0.5767,0.5576,0.5529,0.5504,0.5455
3,https://venturebeat.com/ai/generative-ai-may-b...,Generative AI is going to completely alter the...,0.7521,0.7365,0.7272,0.724,0.7216
4,https://www.businessinsider.com/moderna-ceo-in...,Personalized cancer vaccines could provide a n...,0.5688,0.5227,0.4532,0.4527,0.4452
...,...,...,...,...,...,...,...
111,https://www.foxbusiness.com/personal-finance/h...,"If you’re struggling in the current economy, y...",0.5382,0.5022,0.4537,0.453,0.4166
112,https://www.foxbusiness.com/politics/stuart-va...,"During his latest ""My Take,"" ""Varney & Co."" ho...",0.5482,0.4926,0.4814,0.4076,0.4005
113,https://www.foxbusiness.com/lifestyle/serial-k...,Famed serial killer John Wayne Gacy's creepy c...,0.2994,0.2344,0.2231,0.2163,0.2147
114,https://www.foxbusiness.com/real-estate/us-suf...,The U.S. housing market is experiencing its se...,0.6052,0.5847,0.5802,0.549,0.549


In [6]:
print(df_test.iloc[:, 2:7])

print(len(df_test.index))

        hf1     hf2     hf3     hf4     hf5
0    0.5627  0.5556  0.5153  0.5132  0.4939
1    0.9986  0.6797  0.6685  0.6572  0.6319
2    0.5767  0.5576  0.5529  0.5504  0.5455
3    0.7521  0.7365  0.7272   0.724  0.7216
4    0.5688  0.5227  0.4532  0.4527  0.4452
..      ...     ...     ...     ...     ...
111  0.5382  0.5022  0.4537   0.453  0.4166
112  0.5482  0.4926  0.4814  0.4076  0.4005
113  0.2994  0.2344  0.2231  0.2163  0.2147
114  0.6052  0.5847  0.5802   0.549   0.549
115  0.8797  0.7357  0.6085  0.5983  0.5596

[116 rows x 5 columns]
116


In [7]:
row_list = df_test.loc[79, ['hf1', 'hf2', 'hf3', 'hf4', 'hf5']].values.flatten().tolist()
test_list = list(map(float, row_list))

print(str(test_list))

[1.0, 0.6916, 0.5826, 0.5672, 0.5635]


In [8]:
def removeOutliers(test_list):
    q1 = np.percentile(test_list, 25)
    q3 = np.percentile(test_list, 75)
    iqr = (q3 - q1) * 1.5
    q_set = (q1 - iqr, q3 + iqr)

    result_list =[]

    for num in test_list:
        if num >= q_set[0] and num <= q_set[1]:
            result_list.append(num)

    return result_list

In [9]:
new_list = removeOutliers(test_list)
print(f"Scores with outliers removed: {new_list}")

weighted_score = np.mean(new_list)
print(f"Weighted score: {weighted_score}")

Scores with outliers removed: [0.6916, 0.5826, 0.5672, 0.5635]
Weighted score: 0.601225


In [10]:
def calculateScoreAvg(df):
    score_avg = []
    
    for i in range(len(df)):
        row_list = df.loc[i, ['hf1', 'hf2', 'hf3', 'hf4', 'hf5']].values.flatten().tolist()
        test_list = list(map(float, row_list))
        
        new_list = removeOutliers(test_list)
        weighted_score = np.mean(new_list)
#         print(f"Weighted score of row {i}: {weighted_score}")
        
        score_avg.append(weighted_score)
        i +=1
        
    return score_avg

In [11]:
hf_avg = calculateScoreAvg(df_test)

df_test['hfAvg'] = hf_avg
df_test

,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg
0,https://datainnovation.org/2022/11/visualizing...,The Economist has created a visualization show...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140
1,https://venturebeat.com/ai/stability-ai-looks-...,Among the organizations building generative AI...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325
2,https://venturebeat.com/ai/openai-ceo-admits-c...,Watch on-demand sessions today.\nWatch HereAlt...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600
3,https://venturebeat.com/ai/generative-ai-may-b...,Generative AI is going to completely alter the...,0.7521,0.7365,0.7272,0.724,0.7216,0.732280
4,https://www.businessinsider.com/moderna-ceo-in...,Personalized cancer vaccines could provide a n...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520
...,...,...,...,...,...,...,...,...
111,https://www.foxbusiness.com/personal-finance/h...,"If you’re struggling in the current economy, y...",0.5382,0.5022,0.4537,0.453,0.4166,0.472740
112,https://www.foxbusiness.com/politics/stuart-va...,"During his latest ""My Take,"" ""Varney & Co."" ho...",0.5482,0.4926,0.4814,0.4076,0.4005,0.466060
113,https://www.foxbusiness.com/lifestyle/serial-k...,Famed serial killer John Wayne Gacy's creepy c...,0.2994,0.2344,0.2231,0.2163,0.2147,0.222125
114,https://www.foxbusiness.com/real-estate/us-suf...,The U.S. housing market is experiencing its se...,0.6052,0.5847,0.5802,0.549,0.549,0.573620


In [12]:
df_test.to_json(r'/home/deleidos/Notebooks/JSONData/semanticSimArticlesTestingDF.json')
complete(state="updates written: ")


Cell updates written:  @ 2023-01-19 18:20:42
